# Nutrition5k | Evaluation

In [ ]:
# Standard library imports
from datetime import datetime
from pathlib import Path
from random import seed, shuffle

# Third-party library imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from torchvision import transforms
from torch.utils.data import Dataset
from tqdm import tqdm

# Typing imports
from typing import Any, Dict, List, Tuple

In [ ]:
# set the seed
seed(2024)

In [ ]:
# check whether installed
!gsutil --version

In [ ]:
# download dataset
!mkdir -p nutrition5k_dataset/metadata
!mkdir -p nutrition5k_dataset/dish_ids
!mkdir -p nutrition5k_dataset/imagery/realsense_overhead
!gsutil -m cp -r "gs://nutrition5k_dataset/nutrition5k_dataset/dish_ids/dish_ids_all.txt" ./nutrition5k_dataset/dish_ids/
!gsutil -m cp -r "gs://nutrition5k_dataset/nutrition5k_dataset/metadata/dish_metadata_cafe1.csv" ./nutrition5k_dataset/metadata/
!gsutil -m cp -r "gs://nutrition5k_dataset/nutrition5k_dataset/imagery/realsense_overhead" ./nutrition5k_dataset/imagery/

In [ ]:
!mkdir -p nutrition5k_dataset/dish_ids/splits
!gsutil -m cp -r "gs://nutrition5k_dataset/nutrition5k_dataset/dish_ids/splits/rgb_train_ids.txt" ./nutrition5k_dataset/dish_ids/splits/
!gsutil -m cp -r "gs://nutrition5k_dataset/nutrition5k_dataset/dish_ids/splits/rgb_test_ids.txt" ./nutrition5k_dataset/dish_ids/splits/

In [ ]:
# split original train set in train and validation subsplits
with open("./nutrition5k_dataset/dish_ids/splits/rgb_train_ids.txt") as fp:
  train_ids = fp.read().split("\n")

print(len(train_ids))
cutoff = int(len(train_ids) * 0.8)
shuffle(train_ids)

with open("./nutrition5k_dataset/dish_ids/splits/rgb_test_ids.txt") as fp:
  test_ids = fp.read().split("\n")

print(f"Train/Train Split: {cutoff}, Train/Valid: Split: {len(train_ids)-cutoff}")

with open("./nutrition5k_dataset/dish_ids/splits/rgb_train_train_ids.txt", "w") as fp:
  for i, _id in enumerate(train_ids[:cutoff]):
    if i < cutoff-1:
      fp.write(f"{_id}\n")
    else:
      fp.write(f"{_id}")

with open("./nutrition5k_dataset/dish_ids/splits/rgb_train_val_ids.txt", "w") as fp:
  for i, _id in enumerate(train_ids[cutoff:]):
    if i < cutoff-1:
      fp.write(f"{_id}\n")
    else:
      fp.write(f"{_id}")

In [ ]:
class N5kRealSense(Dataset):
    def __init__(self, path_imagery, path_labels_csv, path_split_txt, transform=None, target_transform=None):
        self.path_imagery = Path(path_imagery)
        assert self.path_imagery.is_dir()

        dish_id_to_image_path = {}
        for path_dish in Path(path_imagery).glob("*"):
          dish_id = path_dish.name
          path_img = Path(path_dish, "rgb.png")
          assert path_img.is_file()
          #print(path_img)
          dish_id_to_image_path[dish_id] = str(path_img)
        self.dish_id_to_image_path = dish_id_to_image_path

        self.labels = pd.read_csv(path_labels_csv, usecols=range(6), header=None, index_col=0)

        with open(path_split_txt, "r") as fp:
          _split_ids = fp.read()
        _split_ids = _split_ids.split("\n")
        self.split_ids = []
        for _split_id in _split_ids:
          if _split_id in self.dish_id_to_image_path:
            self.split_ids.append(_split_id)


        print(f"Split size: {len(self.split_ids)} (orginal: {len(_split_ids)})")

        #self.split_ids = pd.read_csv(path_split_txt, header=None, index_col=None)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.split_ids)

    def __getitem__(self, idx):
        # get next dish id from split list
        dish_id = self.split_ids[idx]
        #print(dish_id)

        # get image for this dish_id
        path_image = self.dish_id_to_image_path[dish_id]
        #assert path_image.is_file(), path_image
        image = Image.open(path_image)
        #image = image.convert("RGB")

        # get label for this dish_id
        target = self.labels.loc[dish_id].to_numpy()

        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            target = self.target_transform(target)

        return image, target

In [ ]:
# generate non-transformed train and validation sets

train_set_no_transform = N5kRealSense(
    path_imagery="./nutrition5k_dataset/imagery/realsense_overhead",
    path_labels_csv="./nutrition5k_dataset/metadata/dish_metadata_cafe1.csv",
    path_split_txt="./nutrition5k_dataset/dish_ids/splits/rgb_train_train_ids.txt",
)

valid_set_no_transform = N5kRealSense(
    path_imagery="./nutrition5k_dataset/imagery/realsense_overhead",
    path_labels_csv="./nutrition5k_dataset/metadata/dish_metadata_cafe1.csv",
    path_split_txt="./nutrition5k_dataset/dish_ids/splits/rgb_train_val_ids.txt",
)

test_set_no_transform = N5kRealSense(
    path_imagery="./nutrition5k_dataset/imagery/realsense_overhead",
    path_labels_csv="./nutrition5k_dataset/metadata/dish_metadata_cafe1.csv",
    path_split_txt="./nutrition5k_dataset/dish_ids/splits/rgb_test_ids.txt",
)

# Example use
# Choose a random sample index
sample_idx = np.random.randint(0, len(train_set_no_transform))

# Get the sample image and target label
image, target = train_set_no_transform[sample_idx]

In [ ]:
def calculate_target_mean(dataset: N5kRealSense) -> Dict[int, float]:
    """
    Calculate the mean across all target dimensions in the input dataset.
    Returns a dictionary of the means.

    Args:
    - dataset (N5kRealSense): The dataset containing target values.

    Returns:
    - Dict[int, float]: A dictionary containing the mean across all target dimensions.
    """

    # Initialize an empty NumPy array to accumulate target values
    target_sum = np.zeros_like(dataset[0][1], dtype=float)

    # Iterate through the dataset to accumulate target values
    for _, target in dataset:
        target_sum += target

    # Calculate the mean across all target dimensions
    num_samples = len(dataset)
    target_mean = {i: value / num_samples for i, value in enumerate(target_sum)}
    target_mean = {
            "total_calories": target_mean[0],
            "total_mass": target_mean[1],
            "total_fat": target_mean[2],
            "total_carb": target_mean[3],
            "total_protein": target_mean[4]}
    return target_mean

target_mean = calculate_target_mean(train_set_no_transform)
target_mean

In [ ]:
def get_target_values(val_set: N5kRealSense) -> np.ndarray:
    """
    Helper function to derive the target values as a NumPy array from the validation set.

    Args:
    - val_set (N5kRealSense): The validation set of the N5kRealSense dataset.

    Returns:
    - np.ndarray: A NumPy array containing the target values from the validation set.
    """
    # Initialize an empty list to store the target values
    target_values = []

    # Iterate through the validation set to extract target values
    for _, target in val_set:
        # Append the target values to the list
        target_values.append(target)

    # Convert the list of target values to a NumPy array
    target_values_np = np.array(target_values)

    return target_values_np

df_test = get_target_values(test_set_no_transform)

In [ ]:
class MeanRegressionModel:
    """Create predictions based on the mean of the training dataset.
    Mean function defined separately"""
    def __init__(self, target_mean: Dict[int, float], val_set: Dataset):
        self.target_mean = target_mean
        self.val_set = val_set

    def predict(self) -> np.ndarray:
        """
        Predict target values for the validation set using the mean dictionary.

        Returns:
        - np.ndarray: Predicted target values for the validation set.
        """
        # Extract the number of dimensions from the mean dictionary
        num_dimensions = len(self.target_mean)

        # Create predictions by repeating the mean values for each sample in the validation set
        predictions = np.array([list(self.target_mean.values()) for _ in range(len(self.val_set))])

        return predictions

mean_regressor = MeanRegressionModel(target_mean, test_set_no_transform)
df_mean_preds = mean_regressor.predict()

In [ ]:
def evaluate_single_output_function(true_values: np.ndarray, predicted_values: np.ndarray) -> Tuple[float, float, float]:
    """
    Evaluate the performance of a single output regression model.

    Args:
    - true_values (np.ndarray): Ground truth target values.
    - predicted_values (np.ndarray): Predicted target values.

    Returns:
    - Tuple[float, float, float]: MAE, MAPE, R2 scores.
    """
    mae = mean_absolute_error(true_values, predicted_values)
    mape = mean_absolute_percentage_error(true_values+0.1, predicted_values) # avoid division by zero
    r2 = r2_score(true_values, predicted_values)
    return mae, mape, r2

def calculate_metrics(target_dims: List[str], true_values: np.ndarray, predicted_values: np.ndarray) -> pd.DataFrame:
    """
    Calculate metrics for each target dimension.

    Args:
    - target_dims (List[str]): List of target dimensions.
    - true_values (np.ndarray): NumPy array containing true target values.
    - predicted_values (np.ndarray): NumPy array containing predicted target values.

    Returns:
    - pd.DataFrame: DataFrame containing calculated metrics.
    """
    metrics_dict = {"dimension": [], "MAE": [], "MAPE": [], "R2": []}

    for dim in target_dims:
        # Translate dimension into index
        dim_index = {"cal": 0, "mass": 1, "protein": 2, "carb": 3, "fat": 4}[dim]

        true_values_dim = true_values[:, dim_index]
        predicted_values_dim = predicted_values[:, dim_index]

        mae, mape, r2 = evaluate_single_output_function(true_values_dim, predicted_values_dim)
        metrics_dict["dimension"].append(dim)
        metrics_dict["MAE"].append(mae)
        metrics_dict["MAPE"].append(mape)
        metrics_dict["R2"].append(r2)

    metrics_df = pd.DataFrame(metrics_dict).transpose()
    metrics_df = metrics_df.iloc[1:4]
    metrics_df.columns = target_dims

    return metrics_df

# Evaluation of simple baseline model:
target_dims = np.array(["cal", "protein", "carb", "fat"])
metrics_df = calculate_metrics(target_dims, df_test, df_mean_preds)
metrics_df

In [ ]:
# Load predictions of vision transformer
df_vit_preds = np.load('best-outputs2.npy')

In [ ]:
# Evaluation of ViT
target_dims = np.array(["cal", "protein", "carb", "fat"])
metrics_vit_df = calculate_metrics(target_dims, df_test, df_vit_preds)
metrics_vit_df